# Personalized correlations PNP3

 *Wake up glucose vs diet the day before*

The idea is that wakeup glucose is not the same on different days for the same person. I want to understand if it is due to the dietal behavior the day before. Also I want to see if associations between the food in the previous day and the wakeup glucose are different for different people. PNP3 cohort is suitable for this purpose because each person had more than 100 days of CGM connections.

I have to start with computing the wakeup glucose for all the people and each day. The method is to use the time between 6 and 7 unless a person logged the food at this time. Also I have to filter out days and people who logged any food after midnight.

In [1]:
import pandas as pd
from LabData.DataLoaders.CGMLoader import CGMLoader
from LabData.DataLoaders.DietLoggingLoader import DietLoggingLoader
import datetime
%matplotlib inline
cgml = CGMLoader()
dll = DietLoggingLoader()

In [2]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [3]:
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.multitest import fdrcorrection

## Calculating wakeup glucose

This function will only suit for pnp3 because it involves adding adjusted glucose from a file outside LabData. 

In [4]:
def calculate_wakeup_glucose_pnp3(time_between = [5,7], study_ids=[3,49]):    

    """Calculates wakeup glucose in the interval given in time_between 
    depending on the breakfast time"""
    
    # Get the cgm df and combine it with adjusted glucose 
    cgmdf = cgml.get_data(study_ids=study_ids).df
    cgmdf = cgml._remove_first_day_of_connections(cgmdf)
    cgmdf = cgmdf.reset_index()
    cgmdf['hour'] = cgmdf.Date.dt.hour
    cgmdf = cgmdf.set_index('Date')
    cgmdf.index = cgmdf.index.tz_localize(None)
    adj_gluc = pd.read_json('/home/elming/Cache/adj_gl.json')
    adj_gluc['ConnectionID'] = adj_gluc['ConnectionID'].astype(str)
    adj_gluc['GlucoseTimestamp'] = pd.to_datetime(adj_gluc['GlucoseTimestamp'])
    adj_gluc = adj_gluc.rename(columns={'GlucoseTimestamp':'Date'})
    adj_gluc = adj_gluc.set_index(['ConnectionID', 'Date'])
    cgm_adj = pd.merge(cgmdf, adj_gluc['GlucoseAdj50N13_Mm'], on=['ConnectionID', 'Date'])
    cgm_adj = cgm_adj.rename(columns={'GlucoseAdj50N13_Mm':'GlucoseAdj'})

    #  Get the log df 
    log = dll.get_data(study_ids=study_ids).df
    logdf = dll.add_nutrients(log, ['energy_kcal'])
    logdf = dll.squeeze_log(logdf)
    logdf = logdf.reset_index()
    logdf['Day'] = logdf['Date'].dt.date
    
    # Filter out beverages with 0 kcal
    logdf = logdf[logdf['energy_kcal'] != 0]
    
    # Filter out days with first meals earlier than 6 am 
    firstmeals = pd.DataFrame(logdf.groupby(['RegistrationCode', 'Day'])['Date'].first().rename('breakfast_ts'))
    firstmeals = firstmeals[(firstmeals['breakfast_ts'].dt.time > datetime.time(6, 0, 0))]
    
    # Dtype handling. After groupby 'Day' is an object, but I need it to be datetime
    firstmeals = firstmeals.reset_index('Day')
    firstmeals['Day'] = pd.to_datetime(firstmeals['Day'])
    firstmeals = firstmeals.set_index('Day', append=True)
    cgm_adj['Day'] = cgm_adj.index.date
    cgm_adj = cgm_adj.set_index(['RegistrationCode', 'Day'])
    
    # Get cgm and firstmeals ts in one df
    cgm_fm = pd.merge(cgm_adj, firstmeals, on=['RegistrationCode', 'Day'])
    
    # Leave cgm timestamps between 5 and 7 only
    cgm_fm = cgm_fm[(cgm_fm['hour'] >= time_between[0]) & (cgm_fm['hour'] < time_between[1])]
    
    # If breakfast was between 6 and 7 then wakeup glucose is a mean value between 5 and 6, otherwise between 6 and 7
    cgm_fm = cgm_fm[((cgm_fm['hour'] == time_between[0]) & (cgm_fm['breakfast_ts'].dt.hour == time_between[0] + 1)) | 
                    ((cgm_fm['hour'] == time_between[0] + 1) & (cgm_fm['breakfast_ts'].dt.hour >= time_between[1]))]
    wakeup_glucose = pd.DataFrame(cgm_fm.reset_index().groupby(['RegistrationCode', 'Day', 'hour'])['GlucoseAdj'].mean().rename(
                                'wakeup_glucose'))
    
    return wakeup_glucose

In [5]:
wakeup_glucose = calculate_wakeup_glucose_pnp3()

In [6]:
wakeup_glucose.head()

wakeup_glucose
RegistrationCode Day        hour                
111527           2017-11-08 5         103.071429
                 2017-11-10 5          98.071429
                 2017-11-11 6         101.321429
                 2017-11-12 6         107.071429
                 2017-11-13 6         106.071429

## Wakeup glucose variability distribution

In [ ]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

def interquartile_range(x):
    return q3(x) - q1(x)

f = {'wakeup_glucose': ['median', 'std', q1,q3, interquartile_range]}

In [ ]:
morn_gluc_distr = wakeup_glucose.groupby('RegistrationCode').agg(f)

In [ ]:
morn_gluc_distr.columns = morn_gluc_distr.columns.droplevel()

In [ ]:
morn_gluc_distr.sample(10)

In [ ]:
fig = plt.figure(figsize=(7,5))
fig.suptitle('Intrapersonal variability of the morning glucose in PNP3', fontsize=18)
ax1 = fig.add_subplot(111)
ax1.set_xlabel('Interquartile range')
morn_gluc_distr['interquartile_range'].plot.hist()

In [ ]:
morn_gluc_distr['interquartile_range'].mean()

### Todo: I should look at this values before and after the intervention and compare changes for the two diets. 

In [ ]:
wakeup_glucose.shape

For how many days did people wear CGM sensor?

In [ ]:
count = wakeup_glucose.groupby('RegistrationCode')['Day'].count()

In [ ]:
count.plot.hist()

Let's take all people with at least 50 days of  connections.

In [ ]:
rc_to_keep = count[count >= 50].index

In [ ]:
len(rc_to_keep)

We have 207 people in total.

## Leave only those people with more than 50 days of connections

In [7]:
def more_than_x_connection_days(wakeup_glucose_df, x=50):
    wakeup_glucose_df = wakeup_glucose_df.reset_index()
    count = wakeup_glucose_df.groupby('RegistrationCode')['Day'].count()
    rc_to_keep = count[count >= 50].index
    # wakeup_glucose = wakeup_glucose.set_index('RegistrationCode').loc[rc_to_keep]
    wakeup_glucose_df = wakeup_glucose_df[wakeup_glucose_df['RegistrationCode'].isin(rc_to_keep)].drop(columns='hour')
    return wakeup_glucose_df

In [8]:
wakeup_glucose = more_than_x_connection_days(wakeup_glucose)

In [9]:
wakeup_glucose.head()

,RegistrationCode,Day,wakeup_glucose
0,111527,2017-11-08,103.071429
1,111527,2017-11-10,98.071429
2,111527,2017-11-11,101.321429
3,111527,2017-11-12,107.071429
4,111527,2017-11-13,106.071429


In [10]:
wakeup_glucose.shape

(25432, 3)

## Dietary features from the day before

In [11]:
def prepare_data_for_corr(wg_df, nutrient_list, study_ids=[3,49], min_cal_per_day=1000, how='total_diet', daysplithours=[6,12,18]):
    
    """Prepare joint df with wg and dietary features from the day before.
    One can choose to correlate with total diet from the perevious day or with diet split into breakfast, lunch and dinner.
    :param wg_df : wakeup_glucose DataFrame
    :param nutrient_list: list of nutrients to select (for the full list see LabData/DataLoaders/Lists/meal_features.txt)
                                e.g. ['energy_kcal', 'carbohydrate_g', 'protein_g', 'totallipid_g']
                                if None, all nutrients are selected
    :param min_cal_per_day:
    :param how: string, can be 'total_diet' or 'split_diet'. If split_diet is passed, then the day will be split into breakfast,
                        lunch and dinner and the nutrients will be calculated accordingly.
    :param daysplithours: list telling how to split the day 
        
    :return data_df with wg and dietary features to perform the correlations with
    """ 
    carbs_cal_per_gram = 4
    fat_cal_per_gram = 9
    prot_cal_per_gram = 4
    
    log = dll.get_data(study_ids=study_ids).df
    logdf = dll.add_nutrients(log, nutrient_list)
    logdf = dll.squeeze_log(logdf)
    logdf = logdf.reset_index()
    logdf['Day'] = logdf['Date'].dt.date
    #Identify days with good log more than min_cal_per_day
    totaldaylog = logdf.drop(columns=['meal_type']).groupby(['RegistrationCode', 'Day']).sum()
    totaldaylog = totaldaylog[totaldaylog['energy_kcal'] >= min_cal_per_day]
    days_to_keep = totaldaylog.index
    logdf = logdf.set_index(['RegistrationCode', 'Day'])
    logdf = logdf.loc[days_to_keep]
    
    if how == 'total_diet':
        # Add derived  features
        totaldaylog['carbs/lipids'] = totaldaylog['carbohydrate_g'] / totaldaylog['totallipid_g']
        # totaldaylog['caloric%carbs'] = totaldaylog['carbohydrate_g'] * carbs_cal_per_gram / totaldaylog['energy_kcal']
        # totaldaylog['caloric%fat'] = totaldaylog['totallipid_g'] * fat_cal_per_gram / totaldaylog['energy_kcal']
        # Change dtype to datetime for merge with wg_df
        totaldaylog = totaldaylog.reset_index('Day')
        totaldaylog['Day'] = pd.to_datetime(totaldaylog['Day'])
        # Add day to the current day for merge with correct wg
        totaldaylog['Day'] = totaldaylog['Day'] + datetime.timedelta(days=1)
        data_for_corr = pd.merge(wg_df, totaldaylog, on=['RegistrationCode', 'Day'])
        
    elif how == 'split_diet':
        logdf['alloc'] = ''
        logdf.loc[(logdf['Date'].dt.time >= datetime.time(daysplithours[0], 0, 0)) & 
              (logdf['Date'].dt.time < datetime.time(daysplithours[1], 0, 0)),'alloc'] = 'b'
        logdf.loc[(logdf['Date'].dt.time >= datetime.time(daysplithours[1], 0, 0)) & 
              (logdf['Date'].dt.time < datetime.time(daysplithours[2], 0, 0)),'alloc'] = 'l'
        logdf.loc[(logdf['Date'].dt.time > datetime.time(daysplithours[2], 0, 0)),'alloc'] = 'd' 
        # We are only interested in  food between 6 and 24. Days where people logged food during the night will be out after merge with wg_df
        logdf = logdf[logdf['alloc'] != '']
        splitlog = logdf.reset_index().groupby(['RegistrationCode', 'Day', 'alloc']).sum()
        splitlog = splitlog.unstack(level=-1)
        splitlog.columns = ['_'.join(splitlog.columns[i]) for i in range(len(splitlog.columns))]
        # Add carbs/lipids ration
        splitlog['carbs/lipids_b'] = splitlog['carbohydrate_g_b'] / splitlog['totallipid_g_b']
        splitlog['carbs/lipids_d'] = splitlog['carbohydrate_g_d'] / splitlog['totallipid_g_d']
        splitlog['carbs/lipids_l'] = splitlog['carbohydrate_g_l'] / splitlog['totallipid_g_l']
        # Change dtype to datetime for merge with wg_df        
        splitlog = splitlog.reset_index('Day')
        splitlog['Day'] = pd.to_datetime(splitlog['Day'])
        # Add day to the current day for merge with correct wg
        splitlog['Day'] = splitlog['Day'] + datetime.timedelta(days=1)
        data_for_corr = pd.merge(wg_df, splitlog, on=['RegistrationCode', 'Day'])
        data_for_corr = data_for_corr.fillna(0)
        
    data_for_corr = data_for_corr.drop(columns='Day')
    return data_for_corr

Dietary features to test the correlations on:
'alcohol_g', 'caffeine_mg', 'carbohydrate_g', 'carbs/fat', 'energy_kcal', 'protein_g', 'sodium_mg', 'sugarstotal_g', 'totaldietaryfiber_g', 'totallipid_g'

In [12]:
nutrient_list = ['caffeine_mg', 'carbohydrate_g', 'energy_kcal', 'protein_g', 'sodium_mg', 'sugarstotal_g', 'totaldietaryfiber_g', 'totallipid_g']

In [14]:
data_corr_total = prepare_data_for_corr(wakeup_glucose, nutrient_list, study_ids=[3,49], 
                                        min_cal_per_day=1000, how='total_diet', daysplithours=[6,12,18])

In [ ]:

data_corr_split = prepare_data_for_corr(wakeup_glucose_test, nutrient_list, study_ids=[3,49], 
                                        min_cal_per_day=1000, how='split_diet', daysplithours=[6,12,18])

In [ ]:
data_corr_total.head()

In [ ]:
data_corr_total.loc['132178'].plot.scatter(x='sodium_mg', y='wakeup_glucose')

## Correlation table

I could also do the same for the night CV. Make function calculate_night_stats(). 

In [15]:
def corr_wg_dietdaybefore(data_corr, method='spearman'):
    
    """Calculate correlations between the wakeup glucose (WG) and the diet the day before.
    :param method: String 'spearman' or 'pearson'
    
    """
    from scipy.stats import pearsonr, spearmanr
    from statsmodels.stats.multitest import fdrcorrection
    
    means = data_corr.groupby('RegistrationCode').mean()
    std = data_corr.groupby('RegistrationCode').std()
    data_corr = data_corr.set_index('RegistrationCode')
    # standartization of the values into z scores
    for rc in data_corr.index.unique():
        data_corr.loc[rc] = (data_corr.loc[rc] - means.loc[rc])/std.loc[rc]
    data_corr = round(data_corr, 4)
    statistics_df = pd.DataFrame(means.stack(level=-1))
    # I don't want to calculate correlation between wg and itself
    slice_column_names = statistics_df.index.levels[-1][statistics_df.index.levels[-1] != 'wakeup_glucose']
    statistics_df = statistics_df.loc[(slice(None), slice_column_names), :]
    
    if method == 'spearman':
        
        statistics_df['rho_' + method] = 0
        statistics_df['pvalue'] = 0
        statistics_df = statistics_df.drop(columns=0)

        for rc in data_corr.index.unique():
            for column in data_corr.drop(columns='wakeup_glucose').columns:
                spearman = spearmanr(data_corr.loc[rc]['wakeup_glucose'], data_corr.loc[rc][column])
                statistics_df.loc[(rc, column), 'rho_' + method] = spearman[0]
                statistics_df.loc[(rc, column), 'pvalue'] = spearman[1]
                
    elif method == 'pearson':

        statistics_df['rho_' + method] = 0
        statistics_df['pvalue'] = 0
        statistics_df = statistics_df.drop(columns=0)

        for rc in data_corr.index.unique():
            for column in data_corr.drop(columns='wakeup_glucose').columns:
                pearson = pearsonr(data_corr.loc[rc]['wakeup_glucose'], data_corr.loc[rc][column])
                statistics_df.loc[(rc, column), 'rho_' + method] = pearson[0]
                statistics_df.loc[(rc, column), 'pvalue'] = pearson[1]
    
    values = {'rho_spearman': 0, 'pvalue': 1}
    statistics_df = statistics_df.fillna(value=values)
    # Add FDR corrected pvalues
    pvalues = statistics_df['pvalue']
    pvalues_corrected = pd.Series(fdrcorrection(pvalues)[1])
    pvalues_corrected.index = statistics_df.index
    statistics_df['pvalue_corrected'] = pvalues_corrected
    
    return statistics_df
    

In [16]:
corr_table_total = corr_wg_dietdaybefore(data_corr_total)

In [ ]:
corr_table_split = corr_wg_dietdaybefore(data_corr_split)

In [ ]:
corr_table_split.isnull().sum()

In [ ]:
corr_table_total.head()

Normality testing method

In [ ]:
# from numpy.random import seed
# from numpy.random import randn
# from scipy.stats import shapiro
# # seed the random number generator
# seed(1)
# # normality test
# for column in data.columns:
#     stat, p = shapiro(data.loc['300747'][column])
#     print('Statistics=%.3f, p=%.3f' % (stat, p))
#     # interpret
#     alpha = 0.05
#     if p > alpha:
#         print(column, ': Sample looks Gaussian (fail to reject H0)')
#     else:
#         print(column, ': Sample does not look Gaussian (reject H0)')

In PNP3 patients probably avoided alcohol due to the diet. I should exclude this feature from the correlation analysis.

In [ ]:
# spearmancorr = data_corr_total.corr(method='spearman')
# spearmancorr

In [ ]:
# sb.heatmap(pearsoncorr, 
#             xticklabels=pearsoncorr.columns,
#             yticklabels=pearsoncorr.columns,
#             cmap='RdBu_r',
#             annot=True,
#             linewidth=0.5)

In [ ]:
# def calculate_pvalues(df):
#     df = df.dropna()._get_numeric_data()
#     dfcols = pd.DataFrame(columns=df.columns)
#     pvalues = dfcols.transpose().join(dfcols, how='outer')
#     for r in df.columns:
#         for c in df.columns:
#             pvalues[r][c] = round(spearmanr(df[r], df[c])[1], 4)
#     return pvalues

# calculate_pvalues(data.loc['111527']) 

In [ ]:
corr_table_split[(corr_table_split['pvalue_corrected'] < 0.05)]

In [ ]:
corr_table_total[(corr_table_total['pvalue_corrected'] < 0.05)]

## Visualize correlations

In [20]:
def plot_corr_heatmap(corr_table, savefig=False, filename=None):
    
        
    import plotly.graph_objects as go
    
    corr_table['significant'] = 0
    corr_table.loc[(corr_table['pvalue'] < 0.01),'significant'] = 1
    corr_table.loc[(corr_table['significant'] == 0), 'rho_spearman'] = None
    signif_corr = corr_table.unstack(level=-1)['rho_spearman']
    signif_corr.columns.name = None
    rc_list = signif_corr.index
    rc_list = ['rc' + item for item in rc_list]
    diet_feat_list = signif_corr.columns
    sign_corr_list = signif_corr.values.tolist()


    fig = go.Figure(data=go.Heatmap(
                       z=sign_corr_list,
                       x=diet_feat_list,
                       y=rc_list,
                       hoverongaps = False))
    fig.update_layout(
        title={
            'text': "Personalized correlations between the wakeup glucose and dietary features from the day before",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title="Dietary features",
        yaxis_title="RegistrationCode"
        )
    fig.show()
    
    if savefig:
        path = '/home/elming/Cache/plotly_figures/' + filename + '.html'
        fig.write_html(path)


In [22]:
plot_corr_heatmap(corr_table_total, savefig=True, filename='corr_total_diet_pnp3_2')

In [21]:
plot_corr_heatmap(corr_table_total)

In [ ]:
corr_table_total['significant'] = 0

In [ ]:
corr_table_total.loc[(corr_table_total['pvalue_corrected'] < 0.05),'significant'] = 1

In [ ]:
statistics_df = statistics_df.fillna(0)

In [ ]:
corr_table_total.loc[(corr_table_total['significant'] == 0), 'rho_spearman'] = None

In [ ]:
signif_corr = corr_table_total.unstack(level=-1)['rho_spearman']

In [ ]:
signif_corr.columns.name = None

In [ ]:
signif_corr.columns

In [ ]:
signif_corr.head()

In [ ]:
rc_list = signif_corr.index
rc_list = ['rc' + item for item in rc_list]
diet_feat_list = signif_corr.columns

sign_corr_list = signif_corr.values.tolist()

In [ ]:
signif_corr.notna().sum()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   z=sign_corr_list,
                   x=diet_feat_list,
                   y=rc_list,
                   hoverongaps = False))
fig.update_layout(
    title={
        'text': "Personalized correlations between the wakeup glucose and dietary features from the day before",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Dietary features",
    yaxis_title="RegistrationCode"
    )
#fig.write_html("/home/elming/Cache/plotly_figures/personalized_corr_pnp3.html")
fig.show()